In [1]:
import numpy as np
import sys
sys.path.append('../just2017plots')
import plotting_resources as res
from astropy.table import Table, vstack, join

rauto_cut = 22.9

In [2]:
slitcat = Table().read('../catalogs/personal_catalogs/slits_phot_zs_cutonslitdist.csv', format='csv')
speccat = slitcat[np.where((slitcat['slit_distance'] < 1.0) &
                           (slitcat['zSpec'] > 0.0) & 
                           (slitcat['zSpec_Q'] == 1))]
cllist = res.load_clusters()
rfmags = Table().read('../catalogs/eazy_colors/ediscs_zldp_zfors_EAzY_outputs.csv', format='csv')
rfmags.rename_column('id', 'ids')



FAILED TO LOAD CLUSTER CMR


# Add the relevant Rest Frame colors to the object catalog
MAG_AB_rfB, MAG_AB_rfU

In [3]:
fulltable = join(speccat, rfmags['ids','MAG_AB_rfB', 'MAG_AB_rfU'], keys='ids', join_type='left')
fulltable['UB_color'] = fulltable['MAG_AB_rfU'] - fulltable['MAG_AB_rfB']

# Recast spec_type as an integer, clean up bad things to be 0

In [4]:
print(set(fulltable['spec_type']))

{'4!!', '1/2', '2/3', '?', '4', '-', '4:', '2:', '1:', '2', '-1', '3/4', '3', '1'}


In [5]:
for row in fulltable:
    if row['spec_type'] == '1/2':
        row['spec_type'] = np.random.choice(['1','2'])
    elif row['spec_type'] == '2/3':
        row['spec_type'] = np.random.choice(['2','3'])
    elif row['spec_type'] == '3/4':
        row['spec_type'] = np.random.choice(['3','4'])
    if row['spec_type'][-1] in [':', '!']:
        row['spec_type'] = row['spec_type'][0]
    if row['spec_type'] in ['-', '?']:
        row['spec_type'] = '-1'
    

fulltable['spec_type_int'] = [int(s) for s in fulltable['spec_type']]
cleanest = fulltable[np.where(fulltable['spec_type_int'] == 1)]
tinybitofOII = fulltable[np.where(fulltable['spec_type_int'] == 2)]
secondpass = vstack((cleanest, tinybitofOII))



In [6]:
print(cleanest.colnames)

['field', 'ids', 'ra', 'dec', 'x', 'y', 'B1', 'V1', 'R1', 'I1', 'z1', 'B2', 'V2', 'R2', 'I2', 'z2', 'B3', 'V3', 'R3', 'I3', 'z3', 'Bauto', 'Vauto', 'Rauto', 'Iauto', 'zauto', 'zLDP5_7', 'Q5_7', 'zphot', 'zphot_errUP', 'zphot_errLO', 'class_StarR', 'zLDP', 'zLDPerr', 'Q', 'zLDP_good', 'slit_distance', 'zSpec', 'zSpec_Q', 'spec_type', 'MAG_AB_rfB', 'MAG_AB_rfU', 'UB_color', 'spec_type_int']


In [15]:
cmrspec = secondpass

# Loop over clusters and find the CMR using the average U-B of members

In [20]:
cllist['cmr'] = np.zeros(len(cllist))
for cl in cllist:
    objs = cmrspec[np.where((cmrspec['field'] == cl['tablename']) &
                            (abs(cmrspec['zSpec']-cl['z']) < 0.02))]
    #objs = cleanest[np.where((abs(cleanest['zSpec']-cl['z']) < 0.02))]
    cl['cmr'] = np.mean(objs['UB_color'])
    print(cl['papername'], len(objs), cl['cmr'])

#with open('personal_catalogs/cluster_cmr.dat', 'w') as fout:
#    fout.write('papername, cmr\n')
#    for cl in cllist:
#        fout.write('{}, {}\n'.format(cl['papername'], cl['cmr']))

cl1018.8-1211 8 1.4303881346720937
cl1037.9-1243 1 1.2487573069907079
cl1037.9-1243a 8 1.034748571792972
cl1040.7-1155 4 1.2815884268983915
cl1054.4-1146 8 1.2435022758978085
cl1054.7-1245 9 1.3053454371905606
cl1059.2-1253 7 1.4456950229092957
cl1103.7-1245a 4 1.1420429563870904
cl1103.7-1245b 0 nan
cl1138.2-1133 0 nan
cl1216.8-1201 10 1.16186986478182
cl1227.9-1138 3 1.0367133886291395
cl1227.9-1138a 0 nan
cl1232.5-1250 10 1.3689413574878948
cl1301.7-1139 3 1.4204385690600556
cl1301.7-1139a 2 1.1539478270308479
cl1353.0-1137 3 1.161745397271415
cl1354.2-1230 1 1.3843587670620465
cl1354.2-1230a 3 1.0424391739086214
cl1411.1-1148 4 1.222310591410868
cl1420.3-1236 4 1.2071521779241117


/Users/matthewkirby/anaconda3/lib/python3.6/site-packages/astropy/table/column.py:932: UserWarning: Warning: converting a masked element to nan.
  self.data[index] = value


In [24]:
cllist['cmr'] = np.zeros(len(cllist))
for cl in cllist:
    allobjs = fulltable[np.where((fulltable['field'] == cl['tablename']) &
                                 (abs(fulltable['zSpec']-cl['z']) < 0.02))]

    # Try cleanest
    for stype in [2,3,4]:
        objs = allobjs[np.where(allobjs['spec_type_int'] <= stype)]
        if len(objs) > 2:
            break
    
    cl['cmr'] = np.mean(objs['UB_color'])
    print(cl['papername'], len(objs), cl['cmr'])

with open('personal_catalogs/cluster_cmr.dat', 'w') as fout:
    fout.write('papername, cmr\n')
    for cl in cllist:
        fout.write('{}, {}\n'.format(cl['papername'], cl['cmr']))

cl1018.8-1211 8 1.4303881346720937
cl1037.9-1243 4 0.845671557635657
cl1037.9-1243a 8 1.034748571792972
cl1040.7-1155 4 1.2815884268983915
cl1054.4-1146 8 1.2435022758978085
cl1054.7-1245 9 1.3053454371905606
cl1059.2-1253 7 1.4456950229092957
cl1103.7-1245a 4 1.1420429563870904
cl1103.7-1245b 4 0.637444426458126
cl1138.2-1133 6 0.5605398800056426
cl1216.8-1201 10 1.16186986478182
cl1227.9-1138 3 1.0367133886291395
cl1227.9-1138a 3 0.7235132488395392
cl1232.5-1250 10 1.3689413574878948
cl1301.7-1139 3 1.4204385690600556
cl1301.7-1139a 3 1.0022101222845592
cl1353.0-1137 3 1.161745397271415
cl1354.2-1230 5 0.9881737877601345
cl1354.2-1230a 3 1.0424391739086214
cl1411.1-1148 4 1.222310591410868
cl1420.3-1236 4 1.2071521779241117


In [95]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
cllist['cmr'] = np.zeros(len(cllist))
kmeans = KMeans(n_clusters=2)

ldpcat = slitcat[np.where((slitcat['Rauto'] < rauto_cut) & 
                          (slitcat['Q'] >= 4) & 
                          (slitcat['slit_distance'] < 1.0))]
ldpcat = join(ldpcat, rfmags['ids','MAG_AB_rfB', 'MAG_AB_rfU'], keys='ids', join_type='left')
ldpcat['UB_color'] = ldpcat['MAG_AB_rfU'] - ldpcat['MAG_AB_rfB']

for cl in cllist:
    allobjs = ldpcat[np.where((ldpcat['field'] == cl['tablename']) &
                                 (abs(ldpcat['zLDP']-cl['z']) < 0.02))]
    data_to_fit = np.vstack((np.zeros(len(allobjs)), allobjs['UB_color'].data.data)).transpose()
    kmeans.fit(data_to_fit)
    m1 = np.mean(allobjs[np.where(kmeans.labels_)]['UB_color'])
    m2 = np.mean(allobjs[np.where(kmeans.labels_*-1 + 1)]['UB_color'])
    cl['cmr'] = max(m1, m2)

    # If KMeans didn't work, use the old strat
    if np.sum(kmeans.labels_) < 3.0:
        print("Old strat for {}".format(cl['papername']))
        allobjs = fulltable[np.where((fulltable['field'] == cl['tablename']) &
                                 (abs(fulltable['zSpec']-cl['z']) < 0.02))]
        for stype in [1,3,4]:
            objs = allobjs[np.where(allobjs['spec_type_int'] <= stype)]
            if len(objs) > 5:
                break
        cl['cmr'] = np.mean(objs['UB_color'])

    
with open('personal_catalogs/cluster_cmr.dat', 'w') as fout:
    fout.write('papername, cmr\n')
    for cl in cllist:
        fout.write('{}, {}\n'.format(cl['papername'], cl['cmr']))

/Users/matthewkirby/anaconda3/lib/python3.6/site-packages/astropy/table/column.py:932: UserWarning: Warning: converting a masked element to nan.
  self.data[index] = value


Old strat for cl1054.4-1146
Old strat for cl1232.5-1250
